In [1]:
# imports
import os
import sys
import time
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from tf_cnnvis import *

np.random.seed(10)

%load_ext autoreload
%autoreload 2

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# helper method to define model
def deepnn():
    
    filename = './cifar/tfrecords/train.tfrecords'
    filename_q = tf.train.string_input_producer([filename], num_epochs=5)
    
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_q)
    features = tf.parse_single_example(serialized_example, features={
        'image': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64),
    })
    
    image = tf.decode_raw(features['image'], tf.int8)
    image.set_shape([3072])
    image = tf.reshape(image, (3, 32, 32))
    image = tf.transpose(image, [1, 2, 0])
    image = tf.cast(image, tf.float32)
    float_image = tf.image.per_image_standardization(image)
    float_image.set_shape([32, 32, 3])
    
    label = tf.cast(features['label'], tf.int32)
    
    images_batch, labels_batch = tf.train.shuffle_batch([float_image, label], batch_size=128, capacity=10000, min_after_dequeue=2000)
    labels_batch = tf.one_hot(labels_batch, 10)
    labels_batch = tf.reshape(labels_batch, (-1, 10))
    x_image = tf.reshape(images_batch, [-1, 3, 32, 32])
    x_image = tf.transpose(x_image, [0, 2, 3, 1])
#     x_image = tf.image.resize_image_with_crop_or_pad(x_image, target_height=32, target_width=32)

    W_conv1 = weight_variable([5, 5, 3, 6])
    b_conv1 = bias_variable([6])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5, 5, 6, 16])
    b_conv2 = bias_variable([16])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

    h_pool2 = max_pool_2x2(h_conv2)

    W_conv3 = weight_variable([5, 5, 16, 10])
    b_conv3 = bias_variable([10])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
    
    h_pool3 = max_pool_2x2(h_conv3)
    
    gap_1 = tf.nn.avg_pool(h_pool3, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME')
    
    dense = tf.reshape(gap_1, [-1, 10])
    
    dense = tf.reshape(dense, [-1, 10])
    return x_image, dense, h_conv1, h_pool1, h_conv2, h_pool2, h_conv3, h_pool3, gap_1, labels_batch


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [3]:
#load graph and data and run training

tf.reset_default_graph()

# defining TF model
x_image, y_conv, h_conv1, h_pool1, h_conv2, h_pool2, h_conv3, h_pool3, gap_1, y_ = deepnn()

print(y_conv.get_shape().as_list())
print(y_.get_shape().as_list())

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_conv, labels = y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver(max_to_keep=100, keep_checkpoint_every_n_hours=1)
sess= tf.Session()

init_op = tf.group(tf.initialize_all_variables(), tf.initialize_local_variables())
sess.run(init_op)

# qr = tf.train.QueueRunner(queue=queue, enqueue_ops=[enqueue_op] * 1, close_op = queue_close)
# tf.train.add_queue_runner(qr)
with sess.as_default():
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
#     threads = qr.create_threads(sess=sess, coord=coord, start=True)
    i = 0
    while not coord.should_stop():
#         image_b, label_b = sess.run([x_image, y_])
#         print(image_b.get_shape().as_list())
#         print(label_b.get_shape().as_list())
#        batch = mnist.train.next_batch(50
        i += 1
        if i%10 == 0:
            saver.save(sess, './cifar_model/model_chkpt', global_step=i)
            train_accuracy = accuracy.eval()
            print("step %d, training accuracy %g"%(i, train_accuracy))
        sess.run([train_step])
    coord.request_stop()
    coord.join(threads)
        
#     x_feed = mnist.test.images.reshape(-1, 784)[:100]
#     y_feed = mnist.test.labels.reshape(-1, 10)[:100]
#     print("test accuracy %g"%accuracy.eval(feed_dict={x: x_feed, y_: y_feed, 
#                                                       }))

#     test_x = mnist.test.images.reshape(-1, 784)[0]
#     test_y = mnist.test.images.reshape(-1, 10)[0]
#     feed_dict = {x: test_x, y:test_y}

[128, 10]
[128, 10]
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.local_variables_initializer` instead.
step 10, training accuracy 0.0859375
step 20, training accuracy 0.109375
step 30, training accuracy 0.046875
step 40, training accuracy 0.101562
step 50, training accuracy 0.109375
step 60, training accuracy 0.0625
step 70, training accuracy 0.0703125
step 80, training accuracy 0.179688
step 90, training accuracy 0.0703125
step 100, training accuracy 0.117188
step 110, training accuracy 0.0859375
step 120, training accuracy 0.09375
step 130, training accuracy 0.148438
step 140, training accuracy 0.125
step 150, training accuracy 0.09375
step 160, training accuracy 0.125
step 170, training accuracy 0.15625
step 180, training accuracy 0.1

KeyboardInterrupt: 